In [ ]:
!pip install tweepy
import tweepy
import csv
import ssl
import time
from requests.exceptions import Timeout, ConnectionError
from requests.packages.urllib3.exceptions import ReadTimeoutError

# code from:
# https://monkeylearn.com/blog/sentiment-analysis-of-twitter/
#     https://www.earthdatascience.org/courses/use-data-open-source-python/intro-to-apis/twitter-data-in-python/

In [ ]:
# Add your Twitter API credentials
<hidden keys>

In [ ]:
# Handling authentication with Twitter
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)

# Create a wrapper for the API provided by Twitter
api = tweepy.API(auth)

# Setting up the keywords, hashtag or mentions we want to listen
keywords = ["Arvind", "Kejriwal"]

# Set the name for CSV file  where the tweets will be saved
filename = "tweets"


# We need to implement StreamListener to use Tweepy to listen to Twitter
class StreamListener(tweepy.StreamListener):

    def on_status(self, status):

        try:
            # saves the tweet object
            tweet_object = status

            # Checks if its a extended tweet (>140 characters)
            if 'extended_tweet' in tweet_object._json:
                tweet = tweet_object.extended_tweet['full_text']
            else:
                tweet = tweet_object.text

            '''Convert all named and numeric character references
            (e.g. &gt;, &#62;, &#x3e;) in the string s to the
            corresponding Unicode characters'''
            tweet = (tweet.replace('&amp;', '&').replace('&lt;', '<')
                     .replace('&gt;', '>').replace('&quot;', '"')
                     .replace('&#39;', "'").replace(';', " ")
                     .replace(r'\u', " "))

            # Save the keyword that matches the stream
            keyword_matches = []
            for word in keywords:
                if word.lower() in tweet.lower():
                    keyword_matches.extend([word])

            keywords_strings = ", ".join(str(x) for x in keyword_matches)

            # Save other information from the tweet
            user = status.author.screen_name
            timeTweet = status.created_at
            source = status.source
            tweetId = status.id
            tweetUrl = "https://twitter.com/statuses/" + str(tweetId)

            # Exclude retweets, too many mentions and too many hashtags
            if not any((('RT @' in tweet, 'RT' in tweet,
                       tweet.count('@') >= 2, tweet.count('#') >= 3))):

                # Saves the tweet information in a new row of the CSV file
                writer.writerow([tweet, keywords_strings, timeTweet,
                                user, source, tweetId, tweetUrl])

        except Exception as e:
            print('Encountered Exception:', e)
            pass


def work():

    # Opening a CSV file to save the gathered tweets
    with open(filename+".csv", 'w') as file:
        global writer
        writer = csv.writer(file)

        # Add a header row to the CSV
        writer.writerow(["Tweet", "Matched Keywords", "Date", "User",
                        "Source", "Tweet ID", "Tweet URL"])

        # Initializing the twitter streap Stream
        try:
            streamingAPI = tweepy.streaming.Stream(auth, StreamListener())
            streamingAPI.filter(track=keywords)

        # Stop temporarily when hitting Twitter rate Limit
        except tweepy.RateLimitError:
            print("RateLimitError...waiting ~15 minutes to continue")
            time.sleep(1001)
            streamingAPI = tweepy.streaming.Stream(auth, StreamListener())
            streamingAPI.filter(track=[keywords])

        # Stop temporarily when getting a timeout or connection error
        except (Timeout, ssl.SSLError, ReadTimeoutError,
                ConnectionError) as exc:
            print("Timeout/connection error...waiting ~15 minutes to continue")
            time.sleep(1001)
            streamingAPI = tweepy.streaming.Stream(auth, StreamListener())
            streamingAPI.filter(track=[keywords])

        # Stop temporarily when getting other errors
        except tweepy.TweepError as e:
            if 'Failed to send request:' in e.reason:
                print("Time out error caught.")
                time.sleep(1001)
                streamingAPI = tweepy.streaming.Stream(auth, StreamListener())
                streamingAPI.filter(track=[keywords])
            else:
                print("Other error with this user...passing")
                pass


if __name__ == '__main__':

    work()

NameError: ignored

In [ ]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)

# Create a wrapper for the API provided by Twitter
api = tw.API(auth)

# Setting up the keywords, hashtag or mentions we want to listen
keywords = ["Arvind", "Kejriwal"]

# Set the name for CSV file  where the tweets will be saved
filename = "tweets"


In [ ]:
search_words ="karan "+" johar "
date_since = "2010-05-28"

tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(50)
[tweet.text for tweet in tweets]

["RT @SRKIANS4EVER: • Aditya Chopra's Biggest Hit\n\n• Karan Johar's Biggest Hit\n\n• Nikhil Adwani's Biggest Hit \n\n• Rohit Shetty's Biggest Hit…",
 "Biggest reality show of India and makers couldn't even manage to fix the poor @justvoot app.\n\nSara paisa Salman aur… https://t.co/zPmbrC9baG",
 'RT @being_battalion: "I remember the impact that Salman had in KKHH." ~ Karan Johar.\n\nMegastar for a reason #SalmanKhan. 🤫 https://t.co/QTS…',
 'RT @Itsss_Shivam: The way Karan johar said - vo toh bahut bade star the🔥,Even Srk said in an interview that Salman was a bigger star than h…',
 'Karan johar has found his toodles partner in the house\n#BiggBoss15\n#BiggBossOTT\n#Karanjohar',
 'Choreographer @nishantbhat85 sets the stage on fire. Karan Johar reveals he was the one to give steps for popular s… https://t.co/OiF6Ou8AoR',
 'RT @Itsss_Shivam: The way Karan johar said - vo toh bahut bade star the🔥,Even Srk said in an interview that Salman was a bigger star than h…',
 'RT @Itsss_Shivam: Th

In [ ]:
new_search = search_words + " -filter:retweets"
tweets = tw.Cursor(api.search, 
                           q=new_search,
                           lang="en",
                           since=date_since).items(500)

# users_locs = [[tweet.user.screen_name, tweet.user.location] for tweet in tweets]
# users_locs
[tweet.text for tweet in tweets]